In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

# BETA HEATMAP

In [ ]:
df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.beta.R1.toShare.csv"))

#df = df.drop(columns=['Unnamed: 0'])

df.siteid = df.siteid.apply(lambda x: x.replace('meta-', '').replace('meta', 'All'))
consistent_c = {
    'age00to25': '0 - 25',
    'age26to49': '26 - 49',
    'age70to79': '70 - 79',
    'age80plus': '80+',
    'sexfemale': 'Female',
    'raceBlack': 'Black',
    'raceAsian': 'Asian',
    'raceHispanic.and.Other': 'Hispanic and Other',
    'albumin': 'Albumin',
    "AST": 'AST',
    "ALT": 'ALT',
    "creatinine": 'Creatinine',
    'CRP': 'CRP',
    'TB': 'Total bilirubin',
    "LYM": 'Lymphocyte count',
    "neutrophil_count" : 'Neutrophil count',
    "WBC" : 'White blood cell',
    'charlson_score': 'Charlson Score'
}

df.variable = df.variable.apply(lambda x: consistent_c[x])

print(df.variable)
#print(df.siteid)
# df = df.rename(columns={"site.from": "from", 'site.to': 'to', 'N.from': 'from_size'})

# site_to_contry = {
#     'APHP': 'France',
#     'FRBDX': 'France',
#     'UKFR': 'France',
#     'BIDMC': 'Germany',
#     'MGB': 'USA',
#     'NWU': 'USA', 
#     'UCLA': 'USA', 
#     'UMICH': 'USA', 
#     'UPENN': 'USA', 
#     'UPITT': 'USA', 
#     'VA1': 'USA', 
#     'VA2': 'USA', 
#     'VA3': 'USA', 
#     'VA4': 'USA', 
#     'VA5': 'USA',
#     'all': 'Meta',
#     'US': 'Meta',
#     'France': 'Meta',
#     'Germany': 'Meta',
# }

# df['country'] = df['from'].apply(lambda x: site_to_contry[x])
# df['to-country'] = df['to'].apply(lambda x: site_to_contry[x])

# df['from'] = df['from'].apply(lambda x: f"{x} →")
# df['to'] = df['to'].apply(lambda x: f"→ {x.replace('all', 'All').replace('US', 'USA')}") # ←


# countries = ['USA', 'France', 'Germany'] # , 'ITALY']
# country_colors = ['#D45E00', '#0072B2', '#029F73'] # , '#B2AA2F']

# # df = pd.melt(df, id_vars=['siteid', 'to'], var_name='day', value_name='value')

unique_sites = df.siteid.unique().tolist()
print(unique_sites)
unique_sites_order=['All', "USA", "France", "Germany", "APHP", "BIDMC", "FRBDX", "MGB", "NWU", "UCLS", "UKFR", "UMICH",
                   "UPITT", "VA1", "VA2", "VA3", "VA4", "VA5"]
# print(len(unique_sites))

unique_vars = df.variable.unique().tolist()
print(unique_vars)



# df.beta = df.beta.apply(lambda x: -3 if x <= -3 else 3 if x >= 3 else x)

# df[df.variable == 'mis_albumin']

In [ ]:
def plot(df=None):
    d = df.copy()
    
    plot = alt.Chart(
        d
    ).mark_square(
        opacity=1
    ).encode(
        x=alt.X("variable:N", title=None, axis=alt.Axis(labelAngle=-55, domain=True, orient='top'), scale=alt.Scale(), sort=unique_vars),
        y=alt.Y("siteid:N", title=None, axis=alt.Axis(labelAngle=0, domain=True), scale=alt.Scale(), sort=unique_sites_order),
#         size=alt.Size('from_size:Q', title='Sample Size', scale=alt.Scale(range=[100, 2000], type='log'), legend=alt.Legend(direction='horizontal', symbolFillColor='lightgray')),
        size=alt.value(600),
        color=alt.Color("beta:Q", title='Coefficient', scale=alt.Scale(scheme='redblue', domain=[-3, 3]), legend=alt.Legend(direction='horizontal', gradientLength=440, gradientThickness=30)),
    ).properties(
        width=750,
        height=700
    )
    
    plot = plot.properties(
#         title={
#             "text": [
#                 f"Transportability Of Cox Regression Model Across Different Sites",
#             ],
#             "dx": 50,
#             "fontSize": 18
#         }
    )
    
    return plot

res = plot(df)

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    subtitle_font_size=18,
#     legend_stroke_color='white'
)

res
save(res,join("..", "result", "R1-beta-matrix.png"), scalefactor=8.0)

